# 1. LeNet

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os

In [2]:
device = torch.device('cuda:0')

In [3]:
# 트레이닝 데이터셋을 다운로드한다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 테스트 데이터셋을 다운로드한다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
def init_cnn(module):
    # Initialize weights for CNNs
    if type(module) == nn.Linear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)
        
class LeNet(nn.Module):
    # The Lenet-5 model
    def __init__(self):
        super(LeNet, self).__init__()
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size = 5, padding = 2), nn.Dropout(),
            nn.AvgPool2d(kernel_size = 2, stride = 2),
            nn.LazyConv2d(16, kernel_size = 5), nn.Dropout(),
            nn.AvgPool2d(kernel_size = 2, stride = 2),
            nn.Flatten(),
            nn.LazyLinear(120), nn.Dropout(),
            nn.LazyLinear(84), nn.Dropout(),
            nn.LazyLinear(10))
    # 포워드 패스    
    def forward(self, x):
        return self.net(x)

In [5]:
print(LeNet())

LeNet(
  (net): Sequential(
    (0): LazyConv2d(0, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Dropout(p=0.5, inplace=False)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): LazyConv2d(0, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Dropout(p=0.5, inplace=False)
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): LazyLinear(in_features=0, out_features=120, bias=True)
    (8): Dropout(p=0.5, inplace=False)
    (9): LazyLinear(in_features=0, out_features=84, bias=True)
    (10): Dropout(p=0.5, inplace=False)
    (11): LazyLinear(in_features=0, out_features=10, bias=True)
  )
)


C:\Users\BMSSA_16\anaconda3\envs\smplify_x4\lib\site-packages\torch\nn\modules\lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
def train_loop(dataloader, model, loss_fn, optimzer, device):
    size = len(dataloader.dataset)
    model.train() # 모델을 훈련 모드로 설정
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X) # 포워드 패스 수행
        loss = loss_fn(pred, y) # CE 연산
        
        optimzer.zero_grad() # 0 으로 초기화
        loss.backward() # 역전파하여 그래디언트 계산
        optimzer.step() # 연산된 그래디언트를 사용해 파라미터를 업데이트
        
        if batch % 100 == 0: # 매 100회차 마다 다음 내용 출력
            loss, current = loss.item(), batch * len(X)
            #print(f'loss: {loss}, [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval() # 모델을 실행 모드로 설정
    
    with torch.no_grad(): # 그래디언트 연산 안함
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X) # 포워드 패스 수행
            test_loss += loss_fn(pred, y) # CE 연산
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 결과 일치하는지 확인
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}% Average Loss: {test_loss:>8f}\n')

In [7]:
def run(device):
    #device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   
    #device = 'cpu'
    print(f"사용할 장치: {device}")

    model = LeNet().to(device)

    learning_rate = 0.001
    batch_size = 64
    epochs = 10


    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f'Epoch {t+1}\n--------------------------------------')
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(train_dataloader, model, loss_fn, device)
    print("Done!")

In [8]:
run(device)

사용할 장치: cuda:0
Epoch 1
--------------------------------------
Test Error: 
 Accuracy: 11.4% Average Loss: 2.279165

Epoch 2
--------------------------------------
Test Error: 
 Accuracy: 16.5% Average Loss: 2.217363

Epoch 3
--------------------------------------
Test Error: 
 Accuracy: 41.4% Average Loss: 1.965970

Epoch 4
--------------------------------------
Test Error: 
 Accuracy: 57.2% Average Loss: 1.304425

Epoch 5
--------------------------------------
Test Error: 
 Accuracy: 61.4% Average Loss: 1.064871

Epoch 6
--------------------------------------
Test Error: 
 Accuracy: 65.4% Average Loss: 0.980866

Epoch 7
--------------------------------------
Test Error: 
 Accuracy: 67.8% Average Loss: 0.922587

Epoch 8
--------------------------------------
Test Error: 
 Accuracy: 69.7% Average Loss: 0.873312

Epoch 9
--------------------------------------
Test Error: 
 Accuracy: 71.0% Average Loss: 0.830697

Epoch 10
--------------------------------------
Test Error: 
 Accuracy: 72.0